In [1]:
%load_ext autoreload
%autoreload 2
%pylab inline
import pandas as pd
import os
import sys
sys.path.append("/home/muncok/DL/projects/sv_system/")

Populating the interactive namespace from numpy and matplotlib


In [2]:
import itertools
import pickle
from protonet.parser import get_parser
from protonet.speech_train import sv_score, init_protonet, init_sv_loaders, init_seed

ModuleNotFoundError: No module named 'protonet.speech_train'

In [1]:
def secToSample(sec):
    return int(16000 * sec)

Populating the interactive namespace from numpy and matplotlib


In [36]:
options = get_parser().parse_args(args=["-cTr=5", "-cVa=5"])

In [37]:
options.n_dct_filters = 40
options.n_mels = 40
options.timeshift_ms = 100
options.data_folder = "/home/muncok/DL/dataset/SV_sets"
options.window_size= 0.025
options.window_stride= 0.010
options.cache_size = 32768
options.input_format = "mfcc"

In [42]:
options.train_manifest = "../manifests/reddots/si_reddots_train_manifest.csv"
options.val_manifest = "../manifests/reddots/si_reddots_train_manifest.csv"

In [43]:
init_seed(options)

In [48]:
input_uttr_sec = 3
options.input_length = secToSample(input_uttr_sec)
options.num_query_val = 10
enroll_length = 15
options.iterations = 100

# sweep conditions
options.num_support_val = int(enroll_length/input_uttr_sec)
options.input = "../models/reddots/simplecnn/si_reddots_0.2s_random.pt"
model = init_protonet(options)
test_uttrs_lens = [0.2, 0.5, 1, 2, 3]
splice_lens = [0.2, 0.5, 1, 2, 3]
conditions = itertools.product(test_uttrs_lens, splice_lens)
results = []
for test_uttrs_l, splice_l in conditions:
    if splice_l > test_uttrs_l: continue
    # fixing the enrollment speech duration
    options.num_test_frames = int(test_uttrs_l/splice_l)
    options.splice_length = secToSample(splice_l)
    _, val_dataloader = init_sv_loaders(options)
    lda = pickle.load(open("../models/lda/lda_voxc_0.2.pkl", "rb"))
    sv_score(options, val_dataloader, model, ["full", "random", "diff", "std"], lda)

../models/reddots/simplecnn/si_reddots_0.2s_random.pt is loaded


100%|██████████| 100/100 [00:15<00:00,  6.58it/s]

eer[full]: 4.62% +- 0.61%, thresh: 0.93064 +- 0.00495
eer[random]: 5.33% +- 0.69%, thresh: 0.91763 +- 0.00435
eer[diff]: 4.74% +- 0.57%, thresh: 0.92285 +- 0.00406
eer[std]: 5.57% +- 0.63%, thresh: 0.92252 +- 0.00431
lda_eer[full]: 5.25% +- 0.56%, thresh: 0.81385 +- 0.01238
lda_eer[random]: 6.63% +- 0.68%, thresh: 0.79134 +- 0.01297
lda_eer[diff]: 5.64% +- 0.54%, thresh: 0.80668 +- 0.01189
lda_eer[std]: 6.22% +- 0.54%, thresh: 0.82093 +- 0.01125



100%|██████████| 100/100 [00:13<00:00,  7.18it/s]


eer[full]: 4.40% +- 0.60%, thresh: 0.93154 +- 0.00418
eer[random]: 5.20% +- 0.63%, thresh: 0.91940 +- 0.00364
eer[diff]: 4.64% +- 0.56%, thresh: 0.91826 +- 0.00365
eer[std]: 5.38% +- 0.59%, thresh: 0.91714 +- 0.00336
lda_eer[full]: 5.48% +- 0.61%, thresh: 0.80916 +- 0.01206
lda_eer[random]: 6.47% +- 0.64%, thresh: 0.78415 +- 0.01111
lda_eer[diff]: 5.28% +- 0.51%, thresh: 0.80230 +- 0.01078
lda_eer[std]: 5.73% +- 0.54%, thresh: 0.81542 +- 0.01209


  0%|          | 0/100 [00:00<?, ?it/s]


ValueError: operands could not be broadcast together with shapes (750,384) (128,) 